In [59]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

SPOTIPY_CLIENT_ID = '046779f681944034a774878424c60ece'
SPOTIPY_CLIENT_SECRET = '562c5ca61e2240d38435faeec33e4840'
SPOTIPY_REDIRECT_URI = 'https://pythonprogramming.net/machine-learning-tutorials/'
SCOPE='user-library-read user-library-modify user-read-private user-read-email user-top-read playlist-read-private playlist-modify-private playlist-modify-public playlist-read-collaborative user-follow-read user-follow-modify user-read-playback-state user-modify-playback-state user-read-currently-playing app-remote-control streaming ugc-image-upload user-read-recently-played user-read-playback-position user-read-playback-state user-modify-playback-state user-library-read user-library-modify user-read-private user-read-email user-top-read playlist-read-private playlist-modify-private playlist-modify-public playlist-read-collaborative user-follow-read user-follow-modify user-read-playback-state user-modify-playback-state user-read-currently-playing app-remote-control streaming ugc-image-upload user-read-recently-played user-read-playback-position'
Auth="https://accounts.spotify.com/authorize"
Token="https://accounts.spotify.com/api/token"

In [106]:
auth_manager = SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                            client_secret=SPOTIPY_CLIENT_SECRET,
                            redirect_uri=SPOTIPY_REDIRECT_URI,
                            scope=SCOPE)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [93]:
song_name = "eminem"
results = sp.search(q=song_name, limit=5, type='track')
songs = [s['uri'] for s in results['tracks']['items']]
print(songs)
sp.start_playback(uris=songs)

['spotify:track:77Ft1RJngppZlq59B6uP0z', 'spotify:track:7lQ8MOhq6IN2w8EYcFNSUk', 'spotify:track:3yfqSUWxFvZELEM4PmlwIR', 'spotify:track:5Z01UMMf7V1o0MzF86s6WJ', 'spotify:track:4xkOaSrkexMciUUogZKVTS']


In [109]:
song_name = "miley cyrus"
results = sp.search(q=song_name, limit=1, type='playlist')
songs = [s['uri'] for s in results['playlists']['items']]
print(songs)
sp.start_playback(context_uri=songs[0])

['spotify:playlist:37i9dQZF1E4FcyO3D0tWcb']


In [113]:
sp.add_to_queue('spotify:track:5xTtaWoae3wi06K5WfVUUH')

In [ ]:
q = sp.queue()
q

In [88]:
sp.start_playback(context_uri='spotify:playlist:7KRBAq1BNjfkTRYsfUHHTk')

In [ ]:
q['currently_playing']

In [114]:
sp.devices()['devices']

[{'id': '2d7e843f0ac01e86cce0b06f4414cc2fa149f2e4',
  'is_active': False,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'Marinus’s MacBook Pro',
  'supports_volume': True,
  'type': 'Computer',
  'volume_percent': 70},
 {'id': '09eb3f4aad1b22bc0a00e982e16887854812ec6e',
  'is_active': True,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'iPhone',
  'supports_volume': False,
  'type': 'Smartphone',
  'volume_percent': 100}]

In [36]:
devices = [d for d in sp.devices()['devices']]
devices

[{'id': '2d7e843f0ac01e86cce0b06f4414cc2fa149f2e4',
  'is_active': False,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'Marinus’s MacBook Pro',
  'supports_volume': True,
  'type': 'Computer',
  'volume_percent': 69},
 {'id': 'b9b59b245707a1c588e6d8b255deea106b1dcfd2',
  'is_active': True,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'Samsung TU7000 55 TV',
  'supports_volume': True,
  'type': 'TV',
  'volume_percent': 10},
 {'id': '09eb3f4aad1b22bc0a00e982e16887854812ec6e',
  'is_active': False,
  'is_private_session': False,
  'is_restricted': False,
  'name': 'iPhone',
  'supports_volume': False,
  'type': 'Smartphone',
  'volume_percent': 100}]

In [53]:
import time
from openai import OpenAI
client = OpenAI()
import json

devices = ['Smartphone', 'TV', 'Computer']
USE = 2
what = "you want it darker"
# chosen_device_id = [d for d in sp.devices()['devices'] if d['type'] == devices[USE]][0]['id']

def is_song_done_playing():
    playback = sp.current_playback()
    try:
      if playback is not None and playback['is_playing']:
          # Check if the progress is close to the total duration (e.g., 95%)
          # print(playback['item'])
          progress_ms = playback['progress_ms']
          duration_ms = playback['item']['duration_ms']
          return progress_ms >= 0.95 * duration_ms
    except:
       return False
    return False

response = client.chat.completions.create(
  model="gpt-3.5-turbo-1106",
  # model="gpt-4-1106-preview",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "You are a helpful assistant designed to output JSON. Json format is as follows '{songs: [name - artist]}'"},
    {"role": "user", "content": what}
  ]
)
# print(response.choices[0].message.content)
print(response.choices[0].message.content)
songs = json.loads(response.choices[0].message.content)["songs"]


for song in songs:
    print(song)
    try:
      results = sp.search(q=song, limit=1, type='track')
      track_uri = results['tracks']['items'][0]['uri']
    except:
       continue
    sp.start_playback(uris=["3BA9yZj3UvOpjjsYOttM2O"], device_id=[d for d in sp.devices()['devices'] if d['type'] == devices[USE]][0]['id'])
    while not is_song_done_playing():
        time.sleep(2) 

KeyboardInterrupt: 